![](https://raw.githubusercontent.com/stack-of-tasks/pinocchio/master/doc/images/pinocchio-logo-large.png)

# Pinocchio: rigib-body derivatives


In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Credits
More than anywhere in Pinocchio, the derivative algorithms are the results of the hard work of Justin Carpentier. Read more about the mathematics behing the code in *Carpentier and Mansard, "Analytical derivatives of rigid body dynamics algorithms", RSS 2018*.

## Set up
We will need Pinocchio, the robot models stored in the package *example-robot-data*, a viewer (either GepettoViewer or MeshCat), some basic linear-algebra operators and the SciPy optimizers.

In [26]:
import pinocchio as pin
import example_robot_data as robex
import numpy as np
from numpy.linalg import inv,pinv,eig,norm,svd,det
from scipy.optimize import fmin_bfgs
import time
import copy
np.set_printoptions(precision=2, linewidth=200, suppress=True)

## Quick description of the tutorial

We will follow the same roadmap as for the previous tutorial, and compute the derivatives of each cost function. We then re-start with a manipulator robot, that has a regular vector configuration (no fancy Lie group there in the begining), and only consider free-basis robot at the very end. 

In [27]:
robot = robex.loadTalosArm()   # Load a 6-dof manipulator arm

Viewer = pin.visualize.GepettoVisualizer
#Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.display(robot.q0)

# jupyter_cell does not like to be inside a if/then block
isinstance(viz,pin.visualize.MeshcatVisualizer) and viz.viewer.jupyter_cell()  ;

In [28]:
rmodel = robot.model
rdata = rmodel.createData()

# Arbitrarily selects a frame (and the parent joint) for later cost functions.
frameIndex = rmodel.getFrameId('gripper_left_fingertip_1_link')
jointIndex = rmodel.frames[frameIndex].parent


## 1. Derivatives of the 6d "placement" cost

We first study the derivatives of the 6d cost. As Pinocchio works with spatial "6d" quantities, this derivative is indeed slightly more intuitive that the one of the 3d cost.

**Notations**: For the derivations that follows, let denote by $\ ^oM_*$ the target placement `Mtarget` and by $\ ^oM_e(q)$ the placement of the operational frame `rdata.oMf[frameIndex]`.

Let's recall first that the 6d cost function is the log of the relative placement $cost(q) = log(^oM_{*}^{-1} \ ^oM_e(q))$, with $\ ^oM_{*}$ a fixed placement, and $\ ^oM_e(q)$ the placement of a given operational frame $\mathcal{F}_e$ of the robot. Applying [the chain rule](https://www.khanacademy.org/math/ap-calculus-ab/ab-differentiation-2-new/ab-3-1a/v/chain-rule-introduction), the derivative of $cost$ must be the product of two derivatives: the derivative of $log$, and the derivative of the relative placement.


In [48]:
# This is a copy of the code explained in the first notebook:
q = pin.randomConfiguration(rmodel)
Mtarget = pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.27])) # arbitrary values
pin.forwardKinematics(rmodel,rdata,q)
Meff = rdata.oMf[frameIndex]
targetMeff = Mtarget.inverse()*Meff
residual = pin.log(targetMeff).vector
cost = sum(residual**2)

### Derivative of the placement of a frame attached to the robot
#### Spatial velocities
The derivative of a place $\ ^AM_B(q)$ is a spatial velocity, denoted $\nu_{AB}$. The spatial velocity is the representation of the vector field of 3D velocity of each point of the rigid body. In Pinocchio, $\nu$ can be represented in two ways:
- either in the $\mathcal{F}_B$ frame, $\ ^B\nu_{AB} = (\ ^Bv_B, \ ^B\omega)$. In that case both the linear part $v$ and the angular part $\omega$ are represented in the $B$ frame, while $v_B$ is the velocity of the center of the frame $\mathcal{F}_B$.
- or in the $\mathcal{F}_A$ frame, $\ ^A\nu_{AB} = (\ ^Av_A, \ ^A\omega)$. In that case, $v$ and $\omega$ are expressed along the frames of $\mathcal{F}_A$, and $v_A$ is the velocity of the point rigidly attached to the body passing through the centre of $\mathcal{F}_A$ at this instant. 

Spatial velocities are implemented in Pinocchio by the class `pin.Motion`, and are respectively argument and output of `pin.exp` and `pin.log`.

#### Joint jacobians
When the spatial velocity of a frame attached to the robot is the consequence of a joint velocity $v_q$, it can be computed as $\nu = J(q) v_q$. As we said, $\nu$ can be represented in an arbitrary frame. The two logical choices are either the `pin.WORLD` frame, i.e. the frame attached to the universe joint `rmodel.joints[0]`; or it can be the local joint attached to the frame we are observing.

Similarly, the Jacobian should be expressed in either of these two frames $\ ^o\nu = \ ^oJ(q) v_q$ or $\ ^e\nu = \ ^eJ(q) v_q$ (where $\ ^oe_E(q)$ is  operational frame of interest).

In [31]:
pin.computeJointJacobians(rmodel,rdata,q)   ### precomputes all jacobians
oJ = pin.getJointJacobian(rmodel,rdata,jointIndex,pin.WORLD)  # in world frame
eJ = pin.getJointJacobian(rmodel,rdata,jointIndex,pin.LOCAL)  # in local frame

The shortcut `pin.computeJacobian(rmodel,rdata,q,jointIndex)` computes a single jacobian, without pre-calculation, but only in the local frame (as running this version of the algorithm in the world frame is not efficient).

#### Frame jacocians
We yet gave the syntax for evaluating the jacobian of a frame attached to a joint. The syntax is quite similar for frame jacobians.

In [33]:
pin.computeJointJacobians(rmodel,rdata,q)   ### precomputes all jacobians
oJf = pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.WORLD)  # in world frame
fJf = pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.LOCAL)

Actually, as the operational frame and the joint space are rigidly attached, their velocity vector fields are the same, hence the expression of their spatial velocity in a same frame are equals, hence their world jacobians are equal.

In [37]:
assert(norm(oJf-oJ)==0)  ### no numerical rounding errors here, they are exactly the same

#### Changing the frame of expression of velocities
If we want to expressed the spatial velocity in another frame, we can move its expression with the corresponding SE3 displacement: change $\ ^A\nu$ expressed in $\mathcal{F}_A$ into $\ ^B\nu$ expressed in $\mathcal{F}_B$ is done with the so-called SE(3) **action**. 

In [40]:
aMb = pin.SE3.Random()
anu = pin.Motion.Random()
bnu = aMb.act(anu)

The SE3 action, also call "big" Adjoint, is a linear operation in $\nu$, that we denote by the action matrix $\ ^AX_B$. The action matrix can be explicited with:

In [42]:
aXb = aMb.action
bnu_vec = aXb @ anu.vector
assert(norm(bnu_vec-bnu.vector)<1e-6)

Jacobians can be changed with the same way. Formally, the colums of the jacobian are spatial velocities, yet they are not represented that way in Pinocchio, and the `pin.SE3.act` function does not work on jacobian. You have to explicitly use the action matrix.

For example, the jacobian of the operation frame wrt the target frame, expressed in the target frame, is $\ ^*J$:

In [ ]:
targetJ = Mtarget.inverse().action @ oJ

If you followed properly, you should be convinced that the jacobian corresponding to $\ ^{target}M_{e}$ is the 
same as the one for $\ ^oM_e$.

#### Log jacobian

Now, we have two representations possible for the jacobian of $(\ ^oM_{*}^{-1} \ ^oM_e)$: either $\ ^*J$ or $\ ^eJ$. Which one should we choose. Well, actually, it depends on the choise we make for representing the jacobian of the log.

M(q) is a function that maps the configuration space (a vector space, so far) into the Lie group SE(3). On the other hand, the *log* is a function thats maps the same Lie group SE(3) into the vector space $\mathbb{R}^6$ (or, more specifically, the space of spatial velocity, which is a real vector space of dimension 6). So, similarly to the jacobian of M, the jacobian of *log* can be represented either in the frame attache to the operational frame $\mathcal{F}_e$ or to the frame attached to the fixed target $\mathcal{F}_*. 

Let's look at the documentation of `pin.Jlog6` to knwo which frame to use:

In [53]:
print(pin.Jlog6.__doc__)


Jlog6( (SE3)Spatial transform M (SE3)) -> object :
    Jacobian of log(M) which maps from the tangent of SE(3) at M to the tangent of SE(3) at Identity.


So, in clear, the frame Jacobian should be expressed in the local frame, as the jacobian of the log is also expressed in this frame.

We can now conclude.

#### Gradrient of the cost function

Our cost is $c(q) = r(q)^T r(q)$, where the residual $r(q) = log(\ ^*M_e(q))$. 
The cost gradient is then $\nabla c = \frac{d c}{d q} = 2 J^T r$, where $J = \frac{dr}{dq}$, the jacobian of the residual, is the product of the jacobian of the log $J_{log}$ and the jacobian of the operational frame placement $J_q$.
$$ \nabla c = 2 J_q^T J_{log}^T r(q)$$

 

![question](question.png)
Rewrite the cost 6d class of the previous notebook, with an additional `def calcDiff(self,q)` function that return the gradient of the cost.

In [62]:
# %load -r 42-72 costs.py
class Cost6d:
    def __init__(self,rmodel,rdata,frameIndex=None, Mtarget=None,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.Mtarget = Mtarget if Mtarget is not None \
                       else pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.27]))  # x,y,z
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes-1
        self.viz = viz

    def residual(self,q):
        '''Compute score from a configuration'''
        pin.forwardKinematics(self.rmodel,self.rdata,q)
        M = pin.updateFramePlacement(self.rmodel,self.rdata,self.frameIndex)
        self.deltaM = self.Mtarget.inverse() * M
        return pin.log(self.deltaM).vector

    def calc(self,q):
        return sum(self.residual(q)**2)

    def callback(self,q):
        if self.viz is None: return
        self.viz.applyConfiguration('world/blue', pin.SE3ToXYZQUATtuple(M))
        self.viz.applyConfiguration('world/box', pin.SE3ToXYZQUATtuple(self.Mtarget))
        self.viz.display(q)
        time.sleep(1e-2)

    def calcDiff(self,q):
        J = pin.computeFrameJacobian(self.rmodel,self.rdata,q,self.frameIndex)
        r = self.residual(q)
        Jlog = pin.Jlog6(self.deltaM)
        return 2 * J.T @ Jlog.T @ r

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>pin.computeJointJacobians(rmodel,rdata,q)</code> precomputes all the jacobians.</li>
        <li><code>pin.getJointJacobian(rmodel,rdata,jointIndex,pin.ReferenceFrame)</code> and <code>pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.ReferenceFrame)</code> returns the joint and frame jacobians, where <code>pin.ReferenceFrame</code> can be either <code>pin.LOCAL</code> or <code>pin.WORLD</code>. </li>
        <li><code>pin.SE3.act</code> can change the expression frame of spatial velocities.</li>
        <li><code>pin.SE3.action</code> is the $6\times 6$ action matrix, that can right multiplies joint and frame jacobians.</li>
        <li><code>pin.Jlog6</code> compute the jacobian of the log of SE(3).</li>
     </ul>
</div>


## 2. Finite differencing for validation

*When you work with gradient based optimization, always start with finite differencing*.  

This sentence could be carved on by grave. There are several reasons to that. First, finite-differences are much easier to implement. It also implies they are less error prone. Most of the time, they work just as well, even if definitely slower. So you can prototype your mathematical program with them, and see if you missed something, at minimal cost. 

And to finish, you *definitely* need to validate your derivatives against finite differencing, as a basic unitary test. 


<div class="alert alert-block alert-danger">
<b>YOU<br>

<b>DEFINITELY<br>

<b>HAVE TO<br>
    </div>

Here is a quick implementation of finite differencing. Use it each time you implement a new derivatives in this tutorial.

In [63]:
def numdiff(func,x,eps=1e-6):
    f0 = copy.copy(func(x))
    xe = x.copy()
    fs = []
    for k in range(len(x)):
        xe[k] += eps
        fs.append( (func(xe)-f0)/eps )
        xe[k] -= eps
    if isinstance(f0,np.ndarray) and len(f0)>1: return np.stack(fs,axis=1)
    else: return np.matrix(fs)

![question](question.png)
Validate your Cost6d.calcDiff with finite differencing.

In [65]:
%do_not_load -r 23-25 solutions.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>assert(  norm(numdiff(cost.calc,q)-cost.calcDiff(q)) < 1e-3)</code> for validating your derivatives (local unefficient implementation).</li>
     </ul>
</div>

## 3. Derivatives the 3d "position" cost

We started with the 6d cost because 6d quantities are more logical in Pinocchio. But we now have nearly everything for differentiating the 3d cost as well. We just need to introduce the `pin.LOCAL_WORLD_ALIGNED` concept.

#### 3d velocities

We consider the position of a point $p$ attached to the robot, expressed in the world frame: $^op(q)$. Its time derivatives corresponds to the velocity of $p$ expressed in the world frame, i.e nothing fancy: $^o\dot{p} = \ ^ov_p$.

Consider first that $p$ is the center of the frame $\mathcal{F}_p$. Then the local expression of the spatial velocity of $\mathcal{F}_p$ is $^p\nu_p = (\ ^pv_p,\ ^p\omega)$, where the linear part $^pv_p$ is the velocity of $p$, expressed in the local frame. We then have:

$$^ov_p = \ ^oR_p \ ^pv_p = \ ^oR_p \ ^p\nu.linear$$

where $^oR_p$ is the rotation of $\mathcal{F}_p$ wrt the world.

#### Local expression aligned to the world

When considering spatial quantities, local or world representation are generally good enough. But when we are interrested by the particular values of its linear part, we are very often in the case described in the previous paragraph: we want the local quantity $v_p$, but we would prefer to have it along world axes, and not the local axes. So jacobians can be evaluated this way. 

In [68]:
oJp = pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.LOCAL_WORLD_ALIGNED)[:3,:]

**Take care**: if you want to consider the 6D jacobian, choosing this representation is very likely to mess something up, as you don't have something matching the spatial algebra anymore. But if you are only interested by the 3D part, this is the way to go. 

Alternatively, you can simply rotate the 3 first rows to align them from local frame to world frame.

In [69]:
oJp2 = rdata.oMf[frameIndex].rotation @ pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.LOCAL)[:3,:]
assert(norm(oJp-oJp2)<1e-6)

#### Cost 3d derivative

Recall first that our 3d cost is $cost(q) = r(q)^T r(q)$ with $r(q) = \ ^op(q)- p_{target}$. 

![question](question.png)

Implement the gradient of the 3d cost introduced in the previous tutorial.

In [ ]:
%do_not_load -r 6-39 costs.py

Don't forget to test it against `numdiff`.

In [75]:
%do_not_load -r 28-30 solutions.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>pin.LOCAL_WORLD_ALIGNED</code> produces spatial quantities expressed locally but represented along the world axes... use it with care!</li>
     </ul>
</div>

## 4. Derivatives of the posture cost

We introduce the posture cost as simple $cost(q) = || q-q^* ||^2$, for a given reference posture $q^*$. Its gradient is straightforward ... let's make it a little bit more difficult.

#### Posture gradient

The cost is $cost(q) = r(q)^T r(q)$ with $r(q)=q-q^*$. The gradient is simply $\frac{d cost}{dq} = 2 r(q)$

#### Posture cost, renewed

When the configuration is a plain vector (i.e. not a fancy Lie element), $q-q^*$ works fine.  But we saw in the last tutorial that it does not work anymore when we have a free basis, hence a quaternion in the configuration. In that case, the residual should be computed by $r(q) = $`pin.difference(rmodel,q,qref)$.

The jacobian of the `pin.difference` operation is given by `pin.dDifference`. This function actually outputs the derivatives with respect to $q$ first, and to $q^*$ second, but we only need the first one, as we consider here that $q^*$ does not vary.

In [79]:
qtarget = robot.q0.copy()
Jdiff,_ = pin.dDifference(rmodel,q,qtarget)

For now, `Jdiff` is identity, but it would not be anymore when we will have a free basis. 

![question](question.png)
Modify the cost posture introduced in the previous notebook, so that it works with `pin.difference`, and add the `CostPosture.calcDiff(self,q)` function to compute its gradient.

In [81]:
# %load -r 133-146 costs.py
class CostPostureDiff:
    def __init__(self,rmodel,rdata,qref=None, viz=None):
        self.rmodel = rmodel
        self.qref = qref if qref is not None else pin.randomConfiguration(rmodel)

    def residual(self,q):
        return pin.difference(self.rmodel,q,self.qref)

    def calc(self,q):
        return sum(self.residual(q)**2)

    def calcDiff(self,q):
        J,_ = pin.dDifference(self.rmodel,q,self.qref)
        return 2*J.T@self.residual(q)

Don't forget to check your gradient against `numdiff`.

In [85]:
# %load -r 33-35 solutions.py
costq = CostPostureDiff(rmodel,rdata)
gq = numdiff(costq.calc,q)
assert(norm(gq-costq.calcDiff(q))<1e-5)

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>pin.dDifference(rmodel,rdata,q1,q2)</code> computes the jacobian of <code>pin.difference</code></li>
     </ul>
</div>

## 5. Derivatives of the two gravity costs

We introduce two cost functions penalizing the gravity cost: $cost_g(q) = g(q)^T g(q)$, and $cost_{Mg}(q) = g(q)^T M(q)^{-1} g(q)$. We will see that the gradient of the first is straightforward, while the gradient of the second involves the derivatives of both RNEA and ABA

#### Gradient of the gravity torque

The gravity $g(q)$ is computed by `pin.computeGeneralizedGravity(rmodel,rdata,q)`. The jacobian of this function is directly implement as `pin.computeGeneralizedGravityDerivatives(rmodel,rdata,q)`.

In [90]:
g = pin.computeGeneralizedGravity(rmodel,rdata,q)
dg = pin.computeGeneralizedGravityDerivatives(rmodel,rdata,q)
dgn = numdiff(lambda q: pin.computeGeneralizedGravity(rmodel,rdata,q),q)
assert(norm(dg-dgn)<1e-4)

The gradient of the gravity cost is simple $\frac{d cost_g}{dq} = 2 \frac{dq}{dq}^T g(q)$.

![question](question.png) 
Copy the gravity cost implemented in the previous tutorial and implement the derivatives of the gravity cost in `CostGravity.calcDiff`.

In [95]:
# %load -r 99-111 costs.py
class CostGravity:
    def __init__(self,rmodel,rdata,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.viz = viz
    def residual(self,q):
        return pin.computeGeneralizedGravity(self.rmodel,self.rdata,q)
    def calc(self,q):
        return sum(self.residual(q)**2)
    def calcDiff(self,q):
        g = self.residual(q)
        G = pin.computeGeneralizedGravityDerivatives(self.rmodel,self.rdata,q)
        return 2*G.T@g

Don't forget ... numdiff ...

In [99]:
# %load -r 38-40 solutions.py
costg = CostGravity(rmodel,rdata)
gg = numdiff(costg.calc,q)
assert(norm(gg-costg.calcDiff(q))/norm(costg.calc(q))<1e-5)

#### Derivatives of the weighted gravity

Let's recall first the RNEA and ABA functions:
$$ rnea(q,v_q,a_q) = \tau_q$$
$$ aba(q,v_q,\tau_q) = a_q$$
The gravity torque can be computed from RNEA when $v_q=0$ and $a_q=0$: $g(q)=rnea(q,v_q=0,a_q=0)$.

Then we have $g(q)^T M(q)^{-1} g(q) = rnea(q,0,0)^T aba(q,0,0)$. To compute its derivatives, the easiest is to rely on the derivatives of RNEA and ABA.

The derivatives of RNEA are computed by `pin.computeRNEADerivatives`. The function computes the derivatives with respect to $q$, $v_q$ and $a_q$, i.e. produces 3 matrices. They are available in `rdata.dtau_dq` and `rdata.dtau_dv`. The derivative wrt to $a_q$ is simply $M(q)$, available in `rdata.M`.

In [107]:
v0 = np.zeros(rmodel.nv)
pin.computeRNEADerivatives(rmodel,rdata,Aq,v0,v0)
assert(norm(rdata.dtau_dq-numdiff(lambda q: pin.rnea(rmodel,rdata,q,v0,v0),q)))

Similarly, the derivatives of ABA are computed by `pin.computeABADerivatives` and stored in `rdata.ddq_dq` and `rdata.ddq_dv`.

In [108]:
pin.computeABADerivatives(rmodel,rdata,q,np.zeros(rmodel.nv),np.zeros(rmodel.nv))
assert(norm(rdata.ddq_dq-numdiff(lambda q: pin.aba(rmodel,rdata,q,v0,v0),q)))

![question](question.png)
Copy the weighted gravity cost implemented in the previous tutorial and implement the derivatives of ths cost in `CostWeightedGravity.calcDiff`.

In [110]:
%do_not_load -r 114-131 costs.py

Don't forget ... numdiff ...

In [ ]:
%do_not_load -r 43-45 solutions.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>pin.computeRNEADerivatives(rmodel,rdata,q,vq,aq)</code> computes the derivatives of RNEA wrt $q$, $v_q$ and $a_q$ and stores them in <code>rdata.dtau_dq</code>, <code>rdata.dtau_dv</code>, <code>rdata.M</code>.</li>
        <li><code>pin.computeABADerivatives(rmodel,rdata,q,vq,tauq)</code> computes the derivatives of ABA wrt $q$, $v_q$ and $\tau_q$ and stores them in <code>rdata.ddq_dq</code>, <code>rdata.ddq_dv</code>, <code>rdata.Minv</code>.</li>
     </ul>
</div>

## 6. The return of the free flyer

At this point, we should already be able to run your previous BFGS program with analytic derivatives. We just need one last step to be able to generalize them to a robot with a free basis.

In [121]:
#robot = robex.loadTalos()
robot = robex.loadANYmal()
viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.display(robot.q0)
isinstance(viz,pin.visualize.MeshcatVisualizer) and viz.viewer.jupyter_cell();

In [126]:
rmodel = robot.model
rdata = rmodel.createData()
frameName = 'wrist_left_ft_tool_link' if rmodel.name == 'talos' else 'LF_FOOT' 
frameIndex = rmodel.getFrameId(frameName)
jointIndex = rmodel.frames[frameIndex].parent

#### Jacobian and tangent application

In the early sections of this tutorial, we have seen that the derivatives in Lie group can go beyond the usual definition. For example, the derivative of the placement, represented by a matrix $4\times 4$, is in fact a vector $6$. These derivatives, that we have called *jacobians*, are indeed *tangent applications*, a notion that matches jacobians when the input and output spaces are vector spaces, but that extends it in Lie groups. To explicit this subtelty, the tangent application of $f(q)$ is sometime denoted $T_qf$, which reads: "the tangent application of f with respect to variable $q$ computed at point $q$" .

When the robot has a free basis, its configuration vector is not a real vector any more, but encompasses a rotation, typically represented by a quaternion in Pinocchio. That $\mathcal{Q}$ is a Lie group!

The derivatives that we get with Pinocchio have the same number of columns as `rmodel.nv`. They must multiply with a velocity $v_q$, and cannot multiply with a vector differene of two configurations $\Delta q$.


In [129]:
q = pin.randomConfiguration(rmodel)
vq = np.random.rand(rmodel.nv)*2-1
J = pin.computeJointJacobian(rmodel,rdata,q,jointIndex)
J@vq  # ok, cartesian velocity of dim 6
try:
    J@q
except:
    print('!!! ERROR')
    print('As expected, you cannot multiply J with a q')

!!! ERROR
As expected, you cannot multiply J with a q


Yet, the solvers from SciPy do not know anything about Lie groups. They work with $q$ as if it was a real vector. Then they expect the derivatives of the cost function as if they were *vector* derivatives. 

In Pinocchio, we call this **coefficient-wise** derivatives. Some coefficient-wise derivatives are implemented, but they are not yet binded in Python. For this tutorial, we propose a Python partial implementation, for the particular case of free-basis robot. Next realeses of Pinocchio would offer an extended, and more efficient solution.

What we need for now is a way of transforming the tangent applications that we already computed, into coefficient-wise jacobians that SciPy is expected.



#### Coefficient-wise derivatives

Let's first recall the notation introduced in the first tutorial: we denote the integration operation with $\oplus$: $ q \oplus v_q = $ `pin.integrate(rmodel,q,vq)`. The tangent application can indeed be defined with $\oplus$ as:
$$ T_qf = \frac{\partial f(q\oplus v_q)}{\partial v_q}$$

By applying the chain rule, we can link the tangent application to the coefficient-wise jacobian. Let' s denote by $h(v_q) := f(q\oplus v_q)$, and by $i(v_q):= q\oplus v_q$. Then $h(v_q) = f \circ i(v_q) = f(i(v_q))$. The chain rule then gives:

$$ T_q f = \frac{df}{dq} \frac{di}{dv_q} $$
where $T_q f$ is the tangent application that we are already computing, $\frac{df}{dq}$ is the coefficient-wise derivative that we are looking for, and $Q := \frac{di}{dv_q}$ is a new matrix that we need to compute. 

#### dExp and dExp_inv

Actually, we need the pseudo inverse of this matrix. Both $di/dv$ and its inverse are implemented in Python in the local final dexp.py


In [136]:
from dexp import dExpQ, dExpQ_inv

Q = dExpQ(rmodel,q)
Qinv = dExpQ_inv(rmodel,q)

#### From tangent application to coefficient-wise jacobian

The tangent $T$ has one less column than the coefficient-wise jacobian $J$. This means that we can pass from $T$ to $J$, but the reciprocal cannot be done without additional prior information. Actually, we can show that $QQ^+$ is a projector onto the normal to the configuration vector, i.e. the only missing information to pass from $T$ to $J$ is "what happen in the direction where the quaternion changes norm$. We then only have an approximation of the coefficient-wise Jacobian, but that is relevant for all the directions that matter.





#### Applying the changes to the gradient algorithm

The gradient that you computed up to now are indeed tangent applications, but you just have to multiply with $Q^+$ to obtain the coefficient-wise that SciPy needs. You can change all the functions above, or (more efficient and less demanding), you can simply apply this final transformation in your mixture of cost.

See the appendix at the end of the tutorial if you need to assert tangent application with finite differences.

![question](question.png)

Copy the sum of costs of the previous tutorial, and implement the `Cost.calcDiff` function by summing the tangent applications (gradients) of the cost already defined and multiplying the result with $Q^+$.

In [167]:
# %load -r 48-69 solutions.py
class SumOfCostFreeFlyer:
    def __init__(self,rmodel,rdata,costs,weights):
        self.rmodel = rmodel
        self.costs = costs
        self.weights = np.array(weights)
    def calc(self,q):
        q = pin.normalize(self.rmodel,q)
        return sum(self.weights*[ cost.calc(q) for cost in self.costs] )
    def callback(self,q):
        q = pin.normalize(self.rmodel,q)
        for c in self.costs:
            if hasattr(c,'callback'): c.callback(q)
    def calcDiff(self,q):
        q = pin.normalize(self.rmodel,q)
        Tqc = sum([ w*cost.calcDiff(q) for (w,cost) in zip(self.weights,self.costs)] )
        Q = dExpQ_inv(self.rmodel,q)
        return Tqc@Q

costsum = SumOfCostFreeFlyer(rmodel,rdata,[ cost3d,costMg ],[1,1e-5])
gsum = numdiff(costsum.calc,q)
assert(norm(gsum-costsum.calcDiff(q))<1e-5)


<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>dExpQ(rmodel,q)</code> and <code>dExpQ_inv(rmodel,q)</code> respectively compute the exponential coefficient-wise derivative and its pseudo inverse.</li>
        <li><code>numdiff(f,q)</code> approximates the coefficient-wise derivative of `f` wrt to `q`.</li>
        <li><code>Tqdiff(f,q)(</code> approximates the Lie tangent application of `f` wrt `q`.</li>
     </ul>
</div>

## Appendix

### Finite differences for tangent applications

If you have chosen not to modify the `Cost.calcDiff` of your main cost classes, then the `numdiff` assertions are not valid any more, as `numdiff` is approximating the coefficient-wise jacobian.
Here is a solution: you can also approximate the tangent application by finite differences, by integrating finite steps instead of vector sums in the `numdif` routine.


In [139]:
def Tdiff(func,exp,nv,q,eps=1e-6):
    """
    Generic finite-differences when the input space is a Lie group, whose integration is defined by q' = q exp(v).
    - func is the function dy differenciate.
    - exp is the integration, working as q2 = exp(q1,vq).
    - nv is the size of the tangent space, i.e. size of vq.
    - q is the point where the tangent application should be evaluated.
    - eps is the finite-difference step.
    """
    f0 = copy.copy(func(q))
    fs = []
    v = np.zeros(nv)
    for k in range(nv):
        v[k] = eps
        qk = exp(q,v)
        fs.append( (func(qk)-f0)/eps )
        v[k] -= eps
    if isinstance(fs[0],np.ndarray) and len(fs[0])>1: return np.stack(fs,axis=1)
    else: return np.array(fs)
    
Tqdiff = lambda func,q: Tdiff(func,exp=lambda q,v: pin.integrate(rmodel,q,v),nv=rmodel.nv,q=q)
    
